# This notebook runs a covariance based Monte Carlo propagation in ADAM

In [1]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import AdamProcessingService
from adam import BatchPropagationResults
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# ConfigManager loads the config set up via adamctl.
# See the README at https://github.com/B612-Asteroid-Institute/adam_home/blob/master/README.md
config = ConfigManager().get_config('adam_dev')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])
aps = AdamProcessingService(auth_rest)

## Example Inputs

The PropagationParameters sent to the propagation API will operate on either Keplerian or Cartesian elements.

In [3]:
# from https://newton.spacedys.com/neodys/index.php?pc=1.1.1.1&n=2000SG344
# true_anomaly_deg is also available instead of mean_anomaly_deg
keplerian_elements = {
    'semi_major_axis_km': 146222900,
    'eccentricity': 0.066958,
    'inclination_deg': 0.112,
    'ra_of_asc_node_deg' : 191.912,
    'arg_of_pericenter_deg' :  275.347,
    'mean_anomaly_deg' : 35.681,
    'gm' : 132712440041.93938 #JPL Horizons GM
}

# Lower triangular covariance matrix (21 elements in a list)
covariance = [3.94346903514E+03, + \
             -1.40266786788E-04, 5.00812620000E-12, + \
             -2.91357694324E-04, 1.06017205000E-11, 3.15658331000E-11, + \
             -3.83826656095E-03, 1.40431472000E-10, 2.32155752000E-09, 8.81161492000E-07, + \
             -1.09220523817E-02, 3.62452521000E-10, -1.53067748000E-09, -8.70304198000E-07, 9.42413982000E-07, + \
             -2.96713683611E-01, 1.05830167000E-08, 2.23110293000E-08, 2.93564832000E-07, 7.81029359000E-07, 2.23721205000E-05]

### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params = PropagationParams({
    'start_time': '2017-10-04T00:00:00Z',   # propagation start time in ISO format
    'end_time': '2017-10-11T00:00:00Z',     # propagation end time in ISO format

    'project_uuid': config['workspace'],
    'description': 'Jupyter Keplerian Covariance Monte Carlo Example',
    'monteCarloDraws': 10,
    'propagationType': 'MONTE_CARLO',
    'stopOnImpact': True,
    'step_size': 86400,
    'stopOnCloseApproach': False,
    'stopOnImpactDistanceMeters': 500000,
    'closeApproachRadiusFromTargetMeters': 7000000000
})

opm_params = OpmParams({
    'epoch': '2017-10-04T00:00:00Z',
    'keplerian_elements': keplerian_elements,
    'keplerian_covariance': covariance,   # object covariance
            
#     'mass': 500.5,              # object mass
#     'solar_rad_area': 25.2,     # object solar radiation area (m^2)
#     'solar_rad_coeff': 1.2,     # object solar radiation coefficient
#     'drag_area': 33.3,          # object drag area (m^2)
#     'drag_coeff': 2.5,          # object drag coefficient
    
})

### Submit and Run Propagation

In [5]:
batch_run = aps.execute_batch_propagation(config['workspace'], propagation_params, opm_params)
# batch_run = BatchPropagationResults.fromRESTwithRawIds(auth_rest, config['workspace'], 'efaa4ecc-c6d0-4d57-a769-cd400a3a0997')
print(batch_run)

{'job_uuid': '933d8fd0-6a39-4bf7-af41-e23af23b0b53', 'results': None}


### Get Status

In [6]:
print(batch_run.check_status())
batch_run.wait_for_complete(print_waiting = True)
print()
print(batch_run.check_status())

CREATED
CREATED
.....RUNNING
...........................
COMPLETED


### Get Number of Runs

In [7]:
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

# Runs: 11


### Get Summary Statistics

In [8]:
stats = batch_run.get_summary()
print(stats)

{'misses': 11, 'close_approach': 0, 'impacts': 0, 'total': 11, 'pc': 0.0}


### Get Ephemeris for specific run

In [9]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

https://storage.googleapis.com/adam-beam-storage/output/job/933d8fd0-6a39-4bf7-af41-e23af23b0b53/stk-ephemerides/MISS/run-5-00001-of-00003.e
stk.v.11.0
BEGIN Ephemeris
ScenarioEpoch 04 Oct 2017 00:00:00.000000
CentralBody SUN
CoordinateSystem ICRF
InterpolationMethod HERMITE
InterpolationOrder 5
NumberOfEphemerisPoints 8

EphemerisTimePosVel
0.000000000000e+00 -4.047751639286e+10 1.302893969618e+11 -2.655266474195e+08 -3.076537454072e+04 -9.557987244399e+03 5.867824146573e+00
8.640000000000e+04 -4.312757684021e+10 1.294382277280e+11 -2.649678323585e+08 -3.057662430384e+04 -1.014435993159e+04 7.067270031341e+00
1.728000000000e+05 -4.576081400241e+10 1.285365668488e+11 -2.643055072210e+08 -3.037596215611e+04 -1.072672981568e+04 8.263797129663e+00
2.592000000000e+05 -4.837620281764e+10 1.275847719264e+11 -2.635399484217e+08 -3.016348486559e+04 -1.130482512352e+04 9.456844895718e+00
3.456000000000e+05 -5.097272690200e+10 1.265832239239e+11 -2.626714807026e+08 -2.993929652470e+04 -1.1878376

### Get ending state vector

In [10]:
end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
if len(end_state_vectors) > 0:
    print(f'First close end state: {end_state_vectors[0]}')
else:
    print("None")


end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
if len(end_state_vectors) > 0:
    print(f'First impact end state: {end_state_vectors[0]}')
else:
    print("None")


end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.MISS)
print("Miss State Vectors")
if len(end_state_vectors) > 0:
    print(f'First miss end state: {end_state_vectors[0]}')
else:
    print("None")



Close State Vectors
None
Impact State Vectors
None
Miss State Vectors
First miss end state: [-200977783778.7074, 81788878462.15994, -18246571187.675217]
